In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
58500,Япония,2020-11-05,104964,94376,1806,1036,812,7
58501,Япония,2020-11-06,106136,95010,1810,1172,634,4
58502,Япония,2020-11-07,107439,95703,1815,1303,693,5
58503,Япония,2020-11-08,108394,96430,1821,955,727,6
58504,Япония,2020-11-09,109191,96684,1834,797,254,13
58505,Япония,2020-11-10,110487,98487,1840,1296,1803,6
58506,Япония,2020-11-11,112011,98305,1847,1524,-182,7
58507,Япония,2020-11-12,113655,99282,1854,1644,977,7
58508,Япония,2020-11-13,115360,100197,1864,1705,915,10
58509,Япония,2020-11-14,117113,101162,1865,1753,965,1


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
298,Весь мир,2020-11-15,54502332,34960714,1319227,473006,229400,6259
299,Весь мир,2020-11-16,55030781,35350741,1327066,528449,390027,7839
300,Весь мир,2020-11-17,55638883,35784056,1338150,608102,433315,11084
301,Весь мир,2020-11-18,56262553,36146977,1349473,623670,362921,11323
302,Весь мир,2020-11-19,56913120,36519821,1360446,650567,372844,10973
303,Весь мир,2020-11-20,57579266,36875075,1372286,666146,355254,11840
304,Весь мир,2020-11-21,58165570,37214376,1381071,586304,339301,8785
305,Весь мир,2020-11-22,58649369,37486362,1388152,483799,271986,7081
306,Весь мир,2020-11-23,59171078,37896435,1396467,521709,410073,8315
307,Весь мир,2020-11-24,59759494,38267861,1409252,588416,371426,12785


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-11-24


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Marshall Islands'), (1, 'Австралия'), (2, 'Австрия'), (3, 'Азербайджан'), (4, 'Албания'), (5, 'Алжир'), (6, 'Ангола'), (7, 'Андорра'), (8, 'Антигуа и Барбуда'), (9, 'Аргентина'), (10, 'Армения'), (11, 'Афганистан'), (12, 'Багамские острова'), (13, 'Бангладеш'), (14, 'Барбадос'), (15, 'Бахрейн'), (16, 'Беларусь'), (17, 'Белиз'), (18, 'Бельгия'), (19, 'Бенин'), (20, 'Болгария'), (21, 'Боливия'), (22, 'Босния и Герцеговина'), (23, 'Ботсвана'), (24, 'Бразилия'), (25, 'Бруней'), (26, 'Буркина-Фасо'), (27, 'Бурунди'), (28, 'Бутан'), (29, 'Вануату'), (30, 'Великобритания'), (31, 'Венгрия'), (32, 'Венесуэла'), (33, 'Весь мир'), (34, 'Восточный Тимор'), (35, 'Вьетнам'), (36, 'Габон'), (37, 'Гаити'), (38, 'Гайана'), (39, 'Гамбия'), (40, 'Гана'), (41, 'Гватемала'), (42, 'Гвинея'), (43, 'Гвинея-Бисау'), (44, 'Германия'), (45, 'Гондурас'), (46, 'Гренада'), (47, 'Греция'), (48, 'Грузия'), (49, 'ДР Конго'), (50, 'Дания'), (51, 'Джибути'), (52, 'Доминика'), (53, 'Доминиканская Республ

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
58823,2020-11-20,128285,107521,1930,2426,1447,10,1.479,1.060,0,0,Япония,18834,1846.428571,1582.071429,17534.000000,14379.428571,44,969
58824,2020-11-21,130871,108981,1932,2586,1460,2,1.635,1.067,0,0,Япония,19958,1965.428571,1673.714286,18372.857143,15096.357143,160,1124
58825,2020-11-22,133034,110273,1943,2163,1292,11,1.502,1.073,0,0,Япония,20818,2060.428571,1760.000000,17806.428571,15858.857143,-423,860
58826,2020-11-23,134554,111343,1949,1520,1070,6,1.200,1.075,0,0,Япония,21262,2142.428571,1811.642857,18720.428571,16615.214286,-643,444
58827,2020-11-24,135786,112385,1963,1232,1042,14,0.859,1.071,0,0,Япония,21438,2077.000000,1807.071429,19558.142857,17420.785714,-288,176


In [6]:
# Добавляем колонки расчитанные ,доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


            Confirmed  Remaining_ill
Date                                
2020-11-23   59171078       19878176
2020-11-24   59759494       20082381
58828
(58828, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
40328,2020-11-05,1699695,Россия,3.488805,2.609639
40329,2020-11-06,1720063,Россия,3.484696,2.572992
40330,2020-11-07,1740172,Россия,3.489305,2.560218
40331,2020-11-08,1760420,Россия,3.490366,2.565242
40332,2020-11-09,1781997,Россия,3.498341,2.593174
40333,2020-11-10,1802762,Россия,3.500897,2.578896
40334,2020-11-11,1822345,Россия,3.495149,2.545867
40335,2020-11-12,1843678,Россия,3.492703,2.499531
40336,2020-11-13,1865395,Россия,3.490938,2.468520
40337,2020-11-14,1887836,Россия,3.494095,2.446961


In [8]:
X.to_csv('W.csv',index=False)